In [ ]:
import os
import numpy as np
from pathlib import Path
from xml.dom.minidom import parse
from shutil import copyfile

In [ ]:
!git clone https://github.com/rkuo2000/yolov5
%cd yolov5

Cloning into 'yolov5'...
remote: Enumerating objects: 12560, done.
remote: Total 12560 (delta 0), reused 0 (delta 0), pack-reused 12560
Receiving objects: 100% (12560/12560), 12.33 MiB | 24.14 MiB/s, done.
Resolving deltas: 100% (8619/8619), done.
/content/yolov5


In [ ]:
!echo "train: Dataset/train/images" > data/alpr.yaml
!echo "val:   Dataset/train/images" >> data/alpr.yaml

!echo "nc : 1" >> data/alpr.yaml
!echo "names: ['license']" >> data/alpr.yaml

!cat data/alpr.yaml

train: Dataset/train/images
val:   Dataset/train/images
nc : 1
names: ['license']


In [ ]:
!pip install -r requirements.txt -q

     |████████████████████████████████| 1.6 MB 5.1 MB/s 


In [ ]:
!unzip /content/drive/MyDrive/datasets/car_plate/archive.zip -d /content/dataset -q

Archive:  /content/drive/MyDrive/datasets/car_plate/archive.zip
  inflating: /content/dataset/annotations/Cars0.xml  
  inflating: /content/dataset/annotations/Cars1.xml  
  inflating: /content/dataset/annotations/Cars10.xml  
  inflating: /content/dataset/annotations/Cars100.xml  
  inflating: /content/dataset/annotations/Cars101.xml  
  inflating: /content/dataset/annotations/Cars102.xml  
  inflating: /content/dataset/annotations/Cars103.xml  
  inflating: /content/dataset/annotations/Cars104.xml  
  inflating: /content/dataset/annotations/Cars105.xml  
  inflating: /content/dataset/annotations/Cars106.xml  
  inflating: /content/dataset/annotations/Cars107.xml  
  inflating: /content/dataset/annotations/Cars108.xml  
  inflating: /content/dataset/annotations/Cars109.xml  
  inflating: /content/dataset/annotations/Cars11.xml  
  inflating: /content/dataset/annotations/Cars110.xml  
  inflating: /content/dataset/annotations/Cars111.xml  
  inflating: /content/dataset/annotations/Cars

In [ ]:
FILE_ROOT = "/content/dataset/"
IMAGE_PATH = FILE_ROOT + "images"  
ANNOTATIONS_PATH = FILE_ROOT + "annotations"

DATA_ROOT = "Dataset/"
DEST_IMAGES_PATH = "train/images"
DEST_LABELS_PATH = "train/labels"

In [ ]:
!mkdir -p Dataset/train/labels

In [ ]:
# copy images
!mkdir -p Dataset/train
!cp -rf /content/dataset/images Dataset/train

In [ ]:
!mkdir -p Dataset/val
!cp -rf /content/dataset/images/Cars1*.png Dataset/val

In [ ]:
def cord_converter(size, box):
    """
    convert xml annotation to darknet format coordinates
    :param size： [w,h]
    :param box: anchor box coordinates [upper-left x,uppler-left y,lower-right x, lower-right y]
    :return: converted [x,y,w,h]
    """
    x1 = int(box[0])
    y1 = int(box[1])
    x2 = int(box[2])
    y2 = int(box[3])

    dw = np.float32(1. / int(size[0]))
    dh = np.float32(1. / int(size[1]))

    w = x2 - x1
    h = y2 - y1
    x = x1 + (w / 2)
    y = y1 + (h / 2)

    x = x * dw
    w = w * dw
    y = y * dh
    h = h * dh
    return [x, y, w, h]
    
def save_file(img_jpg_file_name, size, img_box):
    classes = ['license']
    save_file_name = DATA_ROOT + DEST_LABELS_PATH + '/' + img_jpg_file_name + '.txt'
    print(save_file_name)
    file_path = open(save_file_name, "a+")
    for box in img_box:                  
        #cls_num = classes.index(box[0]) # find class_id
        cls_num = 0
        new_box = cord_converter(size, box[1:]) # convert box coord into YOLO x,y,w,h

        file_path.write(f"{cls_num} {new_box[0]} {new_box[1]} {new_box[2]} {new_box[3]}\n")

    file_path.flush()
    file_path.close()
    
def get_xml_data(file_path, img_xml_file):
    img_path = file_path + '/' + img_xml_file + '.xml'
    print(img_path)

    dom = parse(img_path)
    root = dom.documentElement
    img_name = root.getElementsByTagName("filename")[0].childNodes[0].data
    img_size = root.getElementsByTagName("size")[0]
    objects = root.getElementsByTagName("object")
    img_w = img_size.getElementsByTagName("width")[0].childNodes[0].data
    img_h = img_size.getElementsByTagName("height")[0].childNodes[0].data
    img_c = img_size.getElementsByTagName("depth")[0].childNodes[0].data
    # print("img_name:", img_name)
    # print("image_info:(w,h,c)", img_w, img_h, img_c)
    img_box = []
    for box in objects:
        cls_name = box.getElementsByTagName("name")[0].childNodes[0].data
        x1 = int(box.getElementsByTagName("xmin")[0].childNodes[0].data)
        y1 = int(box.getElementsByTagName("ymin")[0].childNodes[0].data)
        x2 = int(box.getElementsByTagName("xmax")[0].childNodes[0].data)
        y2 = int(box.getElementsByTagName("ymax")[0].childNodes[0].data)
        print("box:(c,xmin,ymin,xmax,ymax)", cls_name, x1, y1, x2, y2)
        img_jpg_file_name = img_xml_file + '.jpg'
        img_box.append([cls_name, x1, y1, x2, y2])
    # print(img_box)
    # test_dataset_box_feature(img_jpg_file_name, img_box)
    save_file(img_xml_file, [img_w, img_h], img_box)

In [ ]:
files = os.listdir(ANNOTATIONS_PATH)
for file in files:
    print("file name: ", file)
    file_xml = file.split(".")
    get_xml_data(ANNOTATIONS_PATH, file_xml[0])

file name:  Cars383.xml
/content/dataset/annotations/Cars383.xml
box:(c,xmin,ymin,xmax,ymax) licence 54 216 88 228
Dataset/train/labels/Cars383.txt
file name:  Cars184.xml
/content/dataset/annotations/Cars184.xml
box:(c,xmin,ymin,xmax,ymax) licence 163 146 212 159
Dataset/train/labels/Cars184.txt
file name:  Cars408.xml
/content/dataset/annotations/Cars408.xml
box:(c,xmin,ymin,xmax,ymax) licence 160 217 187 228
Dataset/train/labels/Cars408.txt
file name:  Cars99.xml
/content/dataset/annotations/Cars99.xml
box:(c,xmin,ymin,xmax,ymax) licence 70 198 262 271
Dataset/train/labels/Cars99.txt
file name:  Cars42.xml
/content/dataset/annotations/Cars42.xml
box:(c,xmin,ymin,xmax,ymax) licence 177 140 217 159
Dataset/train/labels/Cars42.txt
file name:  Cars47.xml
/content/dataset/annotations/Cars47.xml
box:(c,xmin,ymin,xmax,ymax) licence 6 143 48 161
Dataset/train/labels/Cars47.txt
file name:  Cars426.xml
/content/dataset/annotations/Cars426.xml
box:(c,xmin,ymin,xmax,ymax) licence 148 206 282 23

In [ ]:
!mkdir -p Dataset/val/labels
!cp -rf Dataset/train/labels/Cars1*.txt Dataset/val/labels

In [ ]:
!ls Dataset/train/labels

Cars0.txt    Cars166.txt  Cars231.txt  Cars298.txt  Cars363.txt  Cars429.txt
Cars100.txt  Cars167.txt  Cars232.txt  Cars299.txt  Cars364.txt  Cars42.txt
Cars101.txt  Cars168.txt  Cars233.txt  Cars29.txt   Cars365.txt  Cars430.txt
Cars102.txt  Cars169.txt  Cars234.txt  Cars2.txt    Cars366.txt  Cars431.txt
Cars103.txt  Cars16.txt   Cars235.txt  Cars300.txt  Cars367.txt  Cars432.txt
Cars104.txt  Cars170.txt  Cars236.txt  Cars301.txt  Cars368.txt  Cars43.txt
Cars105.txt  Cars171.txt  Cars237.txt  Cars302.txt  Cars369.txt  Cars44.txt
Cars106.txt  Cars172.txt  Cars238.txt  Cars303.txt  Cars36.txt	 Cars45.txt
Cars107.txt  Cars173.txt  Cars239.txt  Cars304.txt  Cars370.txt  Cars46.txt
Cars108.txt  Cars174.txt  Cars23.txt   Cars305.txt  Cars371.txt  Cars47.txt
Cars109.txt  Cars175.txt  Cars240.txt  Cars306.txt  Cars372.txt  Cars48.txt
Cars10.txt   Cars176.txt  Cars241.txt  Cars307.txt  Cars373.txt  Cars49.txt
Cars110.txt  Cars177.txt  Cars242.txt  Cars308.txt  Cars374.txt  Cars4.txt
Cars111.tx

In [ ]:
!python train.py --img 416 --batch 16 --epochs 200 --data data/alpr.yaml --cfg models/yolov5s.yaml

train: weights=yolov5s.pt, cfg=models/yolov5s.yaml, data=data/alpr.yaml, hyp=data/hyps/hyp.scratch-low.yaml, epochs=200, batch_size=16, imgsz=416, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, noplots=False, evolve=None, bucket=, cache=None, image_weights=False, device=, multi_scale=False, single_cls=False, optimizer=SGD, sync_bn=False, workers=8, project=runs/train, name=exp, exist_ok=False, quad=False, cos_lr=False, label_smoothing=0.0, patience=100, freeze=[0], save_period=-1, seed=0, local_rank=-1, entity=None, upload_dataset=False, bbox_interval=-1, artifact_alias=latest
github: up to date with https://github.com/rkuo2000/yolov5 ✅
YOLOv5 🚀 v6.1-303-gb3ca8d8 Python-3.7.13 torch-1.12.1+cu113 CUDA:0 (Tesla T4, 15110MiB)

hyperparameters: lr0=0.01, lrf=0.01, momentum=0.937, weight_decay=0.0005, warmup_epochs=3.0, warmup_momentum=0.8, warmup_bias_lr=0.1, box=0.05, cls=0.5, cls_pw=1.0, obj=1.0, obj_pw=1.0, iou_t=0.2, anchor_t=4.0, fl_gamma=0.0, hsv_h=0.015, hs

In [ ]:
%cd yolov5

/content/yolov5


In [ ]:
!python detect.py --source /content/dataset/images --conf 0.4 --weights runs/train/exp/weights/best.pt --save-crop --save-txt

detect: weights=['runs/train/exp/weights/best.pt'], source=/content/dataset/images, data=data/coco128.yaml, imgsz=[640, 640], conf_thres=0.4, iou_thres=0.45, max_det=1000, device=, view_img=False, save_txt=True, save_conf=False, save_crop=True, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=runs/detect, name=exp, exist_ok=False, line_thickness=3, hide_labels=False, hide_conf=False, half=False, dnn=False
YOLOv5 🚀 v6.1-303-gb3ca8d8 Python-3.7.13 torch-1.12.1+cu113 CUDA:0 (Tesla T4, 15110MiB)

Fusing layers... 
YOLOv5s summary: 213 layers, 7012822 parameters, 0 gradients, 15.8 GFLOPs
image 1/433 /content/dataset/images/Cars0.png: 352x640 1 license, Done. (0.013s)
image 2/433 /content/dataset/images/Cars1.png: 416x640 1 license, Done. (0.012s)
image 3/433 /content/dataset/images/Cars10.png: 384x640 1 license, Done. (0.013s)
image 4/433 /content/dataset/images/Cars100.png: 448x640 1 license, Done. (0.012s)
image 5/433 /content/dataset/i

In [ ]:
!zip -r /content/cropped_images.zip /content/yolov5/runs/detect/exp4

  adding: content/yolov5/runs/detect/exp4/ (stored 0%)
  adding: content/yolov5/runs/detect/exp4/Cars241.png (deflated 3%)
  adding: content/yolov5/runs/detect/exp4/Cars16.png (deflated 2%)
  adding: content/yolov5/runs/detect/exp4/Cars35.png (deflated 1%)
  adding: content/yolov5/runs/detect/exp4/Cars312.png (deflated 10%)
  adding: content/yolov5/runs/detect/exp4/Cars164.png (deflated 1%)
  adding: content/yolov5/runs/detect/exp4/Cars360.png (deflated 1%)
  adding: content/yolov5/runs/detect/exp4/Cars332.png (deflated 1%)
  adding: content/yolov5/runs/detect/exp4/Cars147.png (deflated 4%)
  adding: content/yolov5/runs/detect/exp4/Cars149.png (deflated 2%)
  adding: content/yolov5/runs/detect/exp4/Cars61.png (deflated 1%)
  adding: content/yolov5/runs/detect/exp4/Cars278.png (deflated 1%)
  adding: content/yolov5/runs/detect/exp4/Cars181.png (deflated 2%)
  adding: content/yolov5/runs/detect/exp4/Cars178.png (deflated 1%)
  adding: content/yolov5/runs/detect/exp4/Cars374.png (deflated

In [ ]:
from google.colab import files
files.download("/content/detected_images.zip")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>